Torsional Drillstring Model

@Authors: Ulf Jakob Aarsnes, Roman Shor, Jonathan McIntyre

Copyright 2021 Open Drilling Project

Permission is hereby granted, free of charge, to any person obtaining a copy of this software 
and associated documentation files (the "Software"), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial 
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT 
NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import math
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.integrate import cumtrapz
from parameters_WellB import parameters_WellB
from torsionalWaveStep import torsionalWaveStep

In [2]:
# Parameters
mu = 0.3        # Static Friction
fRat = 0.65     # Ratio between Static and Dynamic Friction
oThres = 1.5    # Omega Threshold
MD = 2500       # Current Measured Depth
p = parameters_WellB(MD, mu, fRat, oThres)
omega_stat = 40 / 60 * 2 * math.pi

In [3]:
fig11 = make_subplots(rows=2, cols=1, vertical_spacing=0.02, shared_xaxes=True)
fig11.add_trace(go.Scatter(x=np.concatenate((p.xp, p.xc)), y=np.concatenate((p.PthetaVec, p.CthetaVec)), line={'color': 'royalblue'}), row=1, col=1)
fig11.add_trace(go.Scatter(x=np.concatenate((p.xp, p.xc)), y=np.concatenate((p.P_thresProfile, p.C_thresProfile)) / 1e3, line={'color': 'royalblue'}), row=2, col=1)
fig11.update_xaxes(title_text='Depth', row=2, col=1)
fig11.update_yaxes(title_text='Inclination (rad)', row=1, col=1)
fig11.update_yaxes(title_text='Coulomb torque per meter (kNm/m)', row=2, col=1)
fig11.update_layout(height=800, width=800, showlegend=False)
fig11.show()

In [4]:
# Basic analytics
# Max stored torque
tauMax = cumtrapz(
    x=np.flip(np.concatenate((p.xp, p.xc))), 
    y=np.flip(-np.concatenate((p.P_thresProfile, p.C_thresProfile))), 
    initial=0
)
tauMax = np.flip(tauMax)

In [5]:
# Twist
phiMax = cumtrapz(
    x=np.flip(np.concatenate((p.xp, p.xc))), 
    y=np.flip(np.concatenate((tauMax[0:p.Pp] / (p.Jp * p.Gp), tauMax[p.Pp:p.Pp + p.Pc] / (p.Jc * p.Gc)))),
    initial=0
)
phiMax = np.flip(phiMax)

phiMax_TD = phiMax[0]
print('phiMax_TD = ', phiMax_TD)

phiMax_TD =  -27.432806491615665


In [6]:
# Amplitude
tauMin = tauMax[0] * (1 - 2 * (1 - fRat))

In [7]:
# Sim stuff
dt = 0.01
T0 = 0
T1 = 60 * 1
t = np.arange(T0, T1 + dt, dt)
Nt = len(t)

# Init vals
op0 = np.ones(p.Pp) * omega_stat * 0
fp0 = np.zeros(p.Pp)
oc0 = np.ones(p.Pc) * omega_stat * 0
fc0 = np.zeros(p.Pc)
Otd0 = 0
# fc0 = np.linspace(2.0e-3 * p.f_tRat, 0, p.Pc)
# fp0 = np.ones(p.Pp) * fc0[0] * p.Jc / p.Jp
# Otd0 = omega_stat

# Parse init states
Pt = p.Pp + p.Pc        # Total number of cells
x0 = np.zeros(2 * Pt + 1)
x0[0:p.Pp] = fp0                # Pipe Strain
x0[p.Pp:Pt] = fc0               # Collar Strain
x0[Pt:Pt+p.Pp] = op0            # Pipe Velocity
x0[Pt+p.Pp:Pt+p.Pp+p.Pc] = oc0  # Collar Velocity
x0[2*Pt] = Otd0                 # Bit Angular position

x = np.zeros((2 * Pt + 1, Nt))
y = np.zeros(Nt)
x[:, 1] = x0

u_wb = 0

Phi_TD = np.zeros((Nt, 12))
omega_sp = 0
II = 0
plotPeriod = 10
# F = math.floor(Nt / plotPeriod)

In [8]:
frames = []
for k in range(Nt - 1):
    omega_sp = omega_sp + (omega_stat - omega_sp) / (1 / dt)

    # Compute Omega Surface
    Otd = x[2*Pt, k]
    u_tau_Motor = (omega_sp - Otd) * 38e3 + II * 100e3
    II += dt * (omega_sp - Otd)

    # Increment the Model
    x[:, k+1] = torsionalWaveStep(p, x[:, k], dt, u_tau_Motor)

    y[k] = u_tau_Motor
    Phi_TD[k+1] = Phi_TD[k] + dt * x[Pt, k]

    if k % plotPeriod == 0:
        yTorque = np.concatenate((x[0:p.Pp, k] * p.Jp * p.Gp, x[p.Pp:Pt, k] * p.Jc * p.Gc)) / 1e3
        yVelocity = x[Pt:2 * Pt, k] * 60 / 2 / math.pi
        frames.append({
            'name': str(round(k * dt, 2)),
            'data': [go.Scatter(x=yTorque), go.Scatter(x=yVelocity)],
            'traces': [0, 2]
        })

In [9]:
fig1 = make_subplots(rows=1, cols=2, horizontal_spacing=0.02, shared_yaxes=True, subplot_titles=('Torque', 'Angular Velocity'))
fig1.add_trace(go.Scatter(x=np.zeros(len(p.xp) + len(p.xc)), y=np.concatenate((p.xp, p.xc)), line={'color': 'royalblue'}), row=1, col=1)
fig1.add_trace(go.Scatter(x=tauMax / 1e3, y=np.concatenate((p.xp, p.xc)), line={'color': 'red'}), row=1, col=1)
fig1.add_trace(go.Scatter(x=np.zeros(len(p.xp) + len(p.xc)), y=np.concatenate((p.xp, p.xc)), line={'color': 'royalblue'}), row=1, col=2)
fig1.update_layout(height=800, width=800, showlegend=False)
fig1.update_yaxes(range=[0, 2500], autorange='reversed')
fig1.update_yaxes(title_text='MD (m)', row=1, col=1)
fig1.update_xaxes(title_text='(kNm)', range=[-3, 25], row=1, col=1)
fig1.update_xaxes(title_text='(RPM)', range=[-7, 150], row=1, col=2)

buttons = [
    {
        'label': 'Play',
        'method': 'animate',
        'args': [
            [frame['name'] for frame in frames],
            {
                'frame': {'duration': 100, 'redraw': False},
                'transition': {'duration': 0},
                'easing': 'linear',
                'fromcurrent': True,
                'mode': 'immediate'
            }
        ]
    },
    {
        "label": "Pause",
        "method": "animate",
        "args": [
            [None],
            {
                "frame": {"duration": 0, "redraw": False},
                "mode": "immediate",
                "transition": {"duration": 0}
            }
        ]
    }
]

updatemenus = [{
    'type':'buttons',
    'buttons': buttons,
    'direction': 'left',
    'pad': {'r': 10, 't': 85},
    'showactive': True,
    'x': 0.1,
    'y': 0,
    'xanchor': 'right',
    'yanchor': 'top'
}]

steps = [{
    'args': [
        [frame['name']], 
        {
            'frame': {'duration': 100.0, 'easing': 'linear', 'redraw': False},
            'transition': {'duration': 0, 'easing': 'linear'}
        }
    ], 
    'label': frame['name'], 
    'method': 'animate'
} for frame in frames]

sliders = [{
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {'font': {'size': 16}, 'prefix': 'Time (s) = ', 'visible': True, 'xanchor': 'right'},
    'transition': {'duration': 100.0, 'easing': 'linear'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': steps
}]

fig1.update(frames=frames)
fig1.update_layout(updatemenus=updatemenus, sliders=sliders)
fig1.show('browser')

In [10]:
fig20 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08, subplot_titles=('Top drive angular velocity, ω<sub>TD</sub>, bit depth = 2500m', 'Motor torque, 𝝉<sub>m</sub>'))
fig20.add_trace(go.Scatter(x=t, y=x[2*Pt, :] * 60 / 2 / math.pi, line={'color': 'royalblue'}), row=1, col=1)
fig20.add_trace(go.Scatter(x=t, y=y / 1e3, line={'color': 'royalblue'}), row=2, col=1)
fig20.add_trace(go.Scatter(x=t, y=t * 0 + tauMax[0] / 1e3, line={'color': 'black', 'dash': 'dash'}), row=2, col=1)
fig20.add_trace(go.Scatter(x=t, y=t * 0 + tauMin / 1e3, line={'color': 'black', 'dash': 'dash'}), row=2, col=1)
fig20.update_xaxes(title_text='Time (s)', row=2, col=1)
fig20.update_yaxes(title_text='(RPM)', row=1, col=1)
fig20.update_yaxes(title_text='(kNm)', row=2, col=1)
fig20.update_layout(height=800, width=800, showlegend=False)
fig20.show()